## 以下采用iexfinance 获取美股股票信息代码

In [488]:
from iexfinance.stocks import Stock
iex_token = "sk_8f26edc50f42482d9d122a1fd4ee34bb"

In [489]:
# 获取开盘价
def get_stock_open(stock_code):
    s = Stock(stock_code, token=iex_token)
    return s.get_open()

In [490]:
# 获取成交量volume
def get_stock_volume(stock_code):
    s = Stock(stock_code, token=iex_token)
    return s.get_volume()

In [491]:
# 获取股票市值
def get_stock_market_cup(stock_code):
    s = Stock(stock_code, token=iex_token)
    return s.get_market_cap()

## 以下采用 openstock 获取美股股票信息

In [5]:
openstock_token = "8d080d3e04efa541"
from openstock.stock.client import StockClient
stock_client = StockClient(openstock_token)

In [6]:
# 获取实时行情
def get_stock_quote(stock_code):
    stock_quote = stock_client.get_stock_quote(query=[stock_code])
    return stock_quote


In [7]:
company_code_maps = {
    'google':'GOOG',
    'apple': 'APPL',
    'alibaba': 'BABA',
    'jd': 'JD',
    'ibm': 'IBM',
    'aws': 'AMZN'
}

def find_stock_code_from_company(company):
    return company_code_maps[company.lower()]



## 使用rasa训练数据

In [9]:
def train(config_file,train_data):
    import warnings
    warnings.filterwarnings('ignore')
    # Import necessary modules
    from rasa.nlu.training_data import load_data
    from rasa.nlu.config import RasaNLUModelConfig
    from rasa.nlu.model import Trainer
    from rasa.nlu import config

    # Create a trainer that uses this config
    trainer = Trainer(config.load(config_file))

    # Load the training data
    training_data = load_data(train_data)

    # Create an interpreter by training the model
    interpreter = trainer.train(training_data)
    return interpreter

interpreter = train("config_spacy.yml","stock-rasa.md")

Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.1s finished
/Users/xixiangweng/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [10]:
# interpreter.parse("i wanna Google open price")
# interpreter.parse("tell me about the volume")
# interpreter.parse("Welcome, I am your robot which can help you stock searching")
interpreter.parse("Sorry, I wanna know something about alibaba instead of jd")

{'intent': {'name': 'stock_search', 'confidence': 0.7004958960422291},
 'entities': [{'start': 36,
   'end': 43,
   'value': 'alibaba',
   'entity': 'company',
   'confidence': 0.8276649055788298,
   'extractor': 'CRFEntityExtractor',
   'processors': ['EntitySynonymMapper']},
  {'start': 55,
   'end': 57,
   'value': 'jd',
   'entity': 'company',
   'confidence': 0.7436892587097529,
   'extractor': 'CRFEntityExtractor'}],
 'intent_ranking': [{'name': 'stock_search', 'confidence': 0.7004958960422291},
  {'name': 'affirm', 'confidence': 0.11025473187407774},
  {'name': 'greet', 'confidence': 0.06838247160573108},
  {'name': 'goodbye', 'confidence': 0.053495856174547815},
  {'name': 'robot_intro', 'confidence': 0.03885847198871904},
  {'name': 'thank', 'confidence': 0.028512572314695346}],
 'text': 'Sorry, I wanna know something about alibaba instead of jd'}

In [11]:
# pattern_prefer = re.compile(".* prefer (?:the)?\s*(\w+) to (?:the)?\s*(\w+)")
# pattern_instead = re.compile(r"(\w+) instead of (\w+)")
# pattern_not = re.compile(".* (\w+), not (?:the)?\s*(\w+)")

# Define negated_ents()
import re
negated_patterns = [
    re.compile(".* prefer (?:the)?\s*(\w+) to (?:the)?\s*(\w+)"),
    re.compile(r"(\w+) instead of (\w+)"),
    re.compile(".* (\w+), not (?:the)?\s*(\w+)")
]

def negated_ents(phrase):
    for pattern in negated_patterns:
        group = pattern.search(phrase)
        if group and (len(group.groups()) >= 2):
            return True,group.groups()

    return False,None


In [12]:
def parse_entities(message):
    entities = interpreter.parse(message)['entities']
    params = {}
    # Fill the dictionary with entities
    for ent in entities:
        params[ent["entity"]] = str(ent["value"])   
    return params

In [13]:
def parse_intent(message):
    intent = interpreter.parse(message)['intent']
    params = {}
    params[intent['name']] = intent['confidence']
    return params

In [14]:
entities = parse_entities('i wanna know APPL open price')
intents = parse_intent('hey, good morning')
print(entities)
print(intents)

{'stock_code': 'appl', 'stock_term': 'open'}
{'greet': 0.6109651688941322}


In [15]:
def find_stock_code_from_message(message):
    entities = parse_entities(message)
    
    if 'company' in entities:
        return find_stock_code_from_company(entities['company'])
    if 'stock_code' in entities:
        return entities['stock_code'].upper()
    return none

In [16]:
find_stock_code_from_message("i wanna APPL open price")

'APPL'

In [17]:
get_stock_quote('AMZN')

[{'bid_size': 0,
  'status': 'NORMAL',
  'high': 1831.091,
  'open': 1828.95,
  'low': 1802.22,
  'symbol': 'AMZN',
  'volume': 2879695,
  'pre_close': 1832.89,
  'latest_time': 1565380800000,
  'ask_size': 0,
  'latest_price': 1807.58,
  'ask_price': 0,
  'bid_price': 0}]

## 多轮会话bot实现

In [18]:
import random
robot_sentences = {
    "robot_intro": [
        "Welcome, I am your robot which can help you stock searching",
        "lalala, I am a robot to help you search for stock information"
    ],
    "specify_company": [
        "So, What kind of stock information would you like to see for {}",
        "~~~ What kind of stock info would you prefer to known for {}",
        "biu biu, so what kind of info would you hope me offer for {}",
    ],
    "reply_open": "Stock open price: {}",
    "reply_volume": "Stock Volume: {}",
    "reply_cap": "Stock Market cap: {}",
    "reply_current": "Current Stock price: {}",
    "reply_all": "Current Stock Info: {}",
    "default": "I'm sorry - I'm not sure how to help you"
}


user_sentences = {
    "ask_open": [
        "what about open price?",
        "how about the open price?"
    ],
    "ask_volume": [
        "what is the stock volume?"
    ],
    "ask_cap": [
        "so the market cap"
    ],
    "ask_all": [
        "tell me {} company current stock station?"
    ],
    "specify_company": [
        "tell me something about {} stock info?"
    ],
    "thank": ["thank you",
              "you are greet"
    ]
}


In [19]:
INIT = 0
ROBOT_INTRO = 1
ASK_COMPANY = 2
ASK_ITEM = 3
ASK_RANDOM = 4
THANK = 5


policy_rules = {
    (INIT, "none"): INIT,
    (INIT, "greet"): ROBOT_INTRO,
    (ASK_RANDOM, "greet"): ASK_COMPANY,
    (ROBOT_INTRO, "specify_company"): ASK_COMPANY,
    (ASK_COMPANY, "specify_company"): ASK_ITEM,
    (ASK_COMPANY, "query_item"): ASK_RANDOM,
    (ASK_ITEM, "query_item"): ASK_RANDOM,
    (ASK_RANDOM, "query_item"): ASK_RANDOM, 
    (ASK_RANDOM, "query_item"): ASK_RANDOM, 
}

In [20]:
# remember state 
class StockMemory(object):
    def __init__(self):
        self.remembered_stock_code = None
        self.remembered_item = None

    def update_remembered_code(self,stock_code):
        self.remembered_stock_code = stock_code

    def get_remembered_code(self):
        return self.remembered_stock_code


    def update_remembered_item(self,query_item):
        self.remembered_item = query_item

    def get_remembered_item(self):
        return self.remembered_item


stockmemory = StockMemory()


import logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                     level=logging.INFO)
## Implementing a state machine
def send_message(policy, state, message):
    print("USER : {}".format(message))
    new_state, response = respond(policy, state, message)
    print("BOT : {}".format(response))
    return new_state

def respond(policy, state, message):
    command = interpret(message)
    new_state = policy[(state, command)]
    response = "enen"
    
    if command == 'none':
        response = robot_sentences['default'] 
    if command == 'greet':
        response = random.choice(robot_sentences['robot_intro'])
        
    if command == 'specify_company':
        response = random.choice(robot_sentences['specify_company']).format(stockmemory.get_remembered_code())
        
    if command == 'query_item':
        item = stockmemory.get_remembered_item()
        quote = get_stock_quote(stockmemory.get_remembered_code())[0]
        if item == 'open price' or item == 'open':
            response = robot_sentences['reply_open'].format(quote['open'])
        if item =='current price' or item == 'current':
            response = robot_sentences['reply_current'].format(quote['latest_price'])
        if item == 'volume':
            response = robot_sentences['reply_volume'].format(quote['volume'])
    return new_state, response

def interpret(message):
    intent = parse_intent(message)
    if 'greet' in intent:
        return 'greet'
    nega_flag, nega_ents = negated_ents(message)
    if nega_flag:
        stockmemory.update_remembered_code(find_stock_code_from_company(nega_ents[0]))
        logging.info("remebered_stock_code: "+ stockmemory.get_remembered_code())
        return 'specify_company'
    
    ents = parse_entities(message)
    if 'stock_code' in ents:
        stockmemory.update_remembered_code(ents['stock_code'])
        logging.info("remebered_stock_code: "+ stockmemory.get_remembered_code())
        return 'specify_company'
    elif 'company' in ents:
        stockmemory.update_remembered_code(find_stock_code_from_company(ents['company']))
        logging.info("remebered_stock_code: "+ stockmemory.get_remembered_code())
        return 'specify_company'
    if 'stock_term' in ents:
        stockmemory.update_remembered_item(ents['stock_term'])
        logging.info("remembered_item: " + stockmemory.get_remembered_item())
        return 'query_item'
    return 'none'

In [21]:
interpreter.parse(random.choice(user_sentences['specify_company']).format('alibaba'))

{'intent': {'name': 'stock_search', 'confidence': 0.834540227440055},
 'entities': [{'start': 24,
   'end': 31,
   'value': 'alibaba',
   'entity': 'company',
   'confidence': 0.9118533491365585,
   'extractor': 'CRFEntityExtractor',
   'processors': ['EntitySynonymMapper']}],
 'intent_ranking': [{'name': 'stock_search', 'confidence': 0.834540227440055},
  {'name': 'greet', 'confidence': 0.03997930093810333},
  {'name': 'robot_intro', 'confidence': 0.03758865236402424},
  {'name': 'affirm', 'confidence': 0.03714374026339388},
  {'name': 'goodbye', 'confidence': 0.029419101727120252},
  {'name': 'thank', 'confidence': 0.021328977267303213}],
 'text': 'tell me something about alibaba stock info?'}

In [22]:
# policy_rules[(0,interpret("hi, good evening"))]
# interpret("i wanna know about the jd company")

In [23]:
send_message(policy_rules, 0, "hi, good evening")

USER : hi, good evening
BOT : lalala, I am a robot to help you search for stock information


1

In [24]:
send_message(policy_rules, 1, "Could you tell me something about jd company")

2019-08-09 21:04:56,861 - root - INFO - remebered_stock_code: JD


USER : Could you tell me something about jd company
BOT : ~~~ What kind of stock info would you prefer to known for JD


2

In [25]:
send_message(policy_rules, 2, "Sorry, I changed for searching alibaba, not the jd")

2019-08-09 21:05:08,752 - root - INFO - remebered_stock_code: BABA


USER : Sorry, I changed for searching alibaba, not the jd
BOT : So, What kind of stock information would you like to see for BABA


3

In [26]:
send_message(policy_rules, 3, "what about the open price")

2019-08-09 21:05:14,938 - root - INFO - remembered_item: open


USER : what about the open price
BOT : Stock open price: 160.98


4

In [27]:
send_message(policy_rules, 3, "tell me today's volume")

2019-08-09 21:05:18,194 - root - INFO - remembered_item: volume


USER : tell me today's volume
BOT : Stock Volume: 11894087


4

## 否定会话测试

In [28]:
tests= [
    "I prefer the google to the apple",
    "Sorry, I wanna know something about alibaba instead of jd",
    "Sorry, I changed for search alibaba, not the jd"
]

In [29]:
b1,items1 = negated_ents(tests[0])
b2,items2 = negated_ents(tests[1])
b3,items3 = negated_ents(tests[2])

In [30]:
b1,items1

(True, ('google', 'apple'))

In [31]:
b2,items2

(True, ('alibaba', 'jd'))

In [518]:
b3,items3

(True, ('alibaba', 'jd'))